## Import Libraries


In [2]:
import os
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")    

## Data Ingestion

In [3]:
import nest_asyncio

# LlamaParse is async-first, so we need to run this line because we are working on a notebook
nest_asyncio.apply()

In [5]:
from llama_parse import LlamaParse
documents = LlamaParse(result_type="markdown").load_data("./data/1_page_2025_Tucson_Hybrid_user_manual.pdf")

Started parsing the file under job_id 9e94fe7e-75cf-4ba4-aa72-767e60e20bf5


# Setting up VectorStoreIndex

## LLM and Embedding Model

In [6]:
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama-3.2-3b-preview", api_key=GROQ_API_KEY)

Settings.llm = llm
Settings.embed_model = embed_model

## Initialize Vector Store Index

In [7]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.67s/it]


## RAG put into practice: Query engine

In [8]:
query_engine = vector_index.as_query_engine()

res = query_engine.query("under what circumstances the navigation-based smart cruise control may not operate properly for the 2025 Tucson Hybrid?")

In [9]:
from IPython.display import Markdown, display

display(Markdown(res.response))

The navigation-based Smart Cruise Control may not operate properly for the 2025 Tucson Hybrid under the following circumstances:

- The navigation is not working properly
- Map information is not transmitted due to infotainment system's abnormal operation
- Speed limit and road information in the navigation is not updated
- The map information and the actual road is different because of real-time GPS data or map information error
- The navigation searches for a route while driving
- GPS signals are blocked in areas such as a tunnel
- A road that divides into two or more roads and joins again
- The driver goes off course from the route set in the navigation
- The route to the destination is changed or cancelled by resetting the navigation
- The vehicle enters a service station or rest area
- Android Auto or Car Play is operating
- The navigation cannot detect the current vehicle position (for example, elevated roads including overpass adjacent to general roads or nearby roads exist in a parallel way)
- The navigation is being updated while driving
- The navigation is being restarted while driving
- The speed limit of some sections changes according to the road situations
- Driving on a road under construction
- Driving in lane-restricted driving situations
- There is bad weather, such as heavy rain, heavy snow, etc.
- Driving on a road that is sharply curved

# Persist

In [10]:
PERSIST_DIR = "./storage"

if not os.path.exists(PERSIST_DIR):
    documents = LlamaParse(result_type="markdown").load_data("./2025_Tucson_Hybrid_user_manual.pdf")
    vector_index = VectorStoreIndex.from_documents(documents)

    # Store Vector Index
    vector_index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing Vector Index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    vector_index = load_index_from_storage(storage_context)

query_engine = vector_index.as_query_engine()
res = query_engine.query("under what circumstances the navigation-based smart cruise control may not operate properly for the 2025 Tucson Hybrid?")

display(Markdown(res.response))

The navigation-based smart cruise control may not operate properly for the 2025 Tucson Hybrid under the following circumstances:

- The navigation is not working properly
- Map information is not transmitted due to infotainment system's abnormal operation
- Speed limit and road information in the navigation is not updated
- The map information and the actual road is different because of real-time GPS data or map information error
- The navigation searches for a route while driving
- GPS signals are blocked in areas such as a tunnel
- A road that divides into two or more roads and joins again
- The driver goes off course from the route set in the navigation
- The route to the destination is changed or cancelled by resetting the navigation
- The vehicle enters a service station or rest area
- Android Auto or Car Play is operating
- The navigation cannot detect the current vehicle position (for example, elevated roads including overpass adjacent to general roads or nearby roads exist in a parallel way)
- The navigation is being updated while driving
- The navigation is being restarted while driving
- The speed limit of some sections changes according to the road situations
- Driving on a road under construction
- Driving in lane-restricted driving situations
- There is bad weather, such as heavy rain, heavy snow, etc.
- Driving on a road that is sharply curved